<a href="https://colab.research.google.com/github/kaisarfardin6620/FlowerNet-Comparison/blob/main/flower_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import shutil
import os
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import logging
from PIL import Image
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.utils import to_categorical
from itertools import cycle

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

In [ ]:
print("--- Mounting Google Drive ---")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}. Please ensure you are in a Colab environment and have granted permissions.")
    exit()

In [ ]:
base_path = '/content/drive/MyDrive/Dataset/flower_images_split'
output_dir = '/content/drive/MyDrive/Dataset/flower_images_split'
os.makedirs(output_dir, exist_ok=True)
print(f"Base dataset path: {base_path}")
print(f"Output directory: {output_dir}")

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
CHANNEL_NUM = 3
BATCH_SIZE = 32
RANDOM_SEED = 42
EPOCHS = 100
input_shape = (IMG_WIDTH, IMG_HEIGHT, CHANNEL_NUM)

print(f"Image dimensions: {IMG_WIDTH}x{IMG_HEIGHT}x{CHANNEL_NUM}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Training epochs: {EPOCHS}")

In [ ]:
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [ ]:
def prepare_and_analyze_dataset(base_path, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):

    print("\n--- Dataset Preparation and Overview ---")

    if not os.path.isdir(base_path):
        print(f"Error: Base path '{base_path}' does not exist. Please check your Drive path.")
        return

    train_path = os.path.join(base_path, 'train')
    val_path = os.path.join(base_path, 'val')
    test_path = os.path.join(base_path, 'test')

    has_train = os.path.isdir(train_path)
    has_val = os.path.isdir(val_path)
    has_test = os.path.isdir(test_path)

    if not has_train and not has_val and not has_test:
        print("No 'train', 'val', 'test' folders found. Splitting raw class folders into splits...")
        class_folders = [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]

        if not class_folders:
            print(f"Error: No class folders found in '{base_path}'. Cannot split dataset.")
            return

        os.makedirs(train_path, exist_ok=True)
        os.makedirs(val_path, exist_ok=True)
        os.makedirs(test_path, exist_ok=True)

        for class_name in class_folders:
            class_source_path = os.path.join(base_path, class_name)
            images = [f for f in os.listdir(class_source_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            random.shuffle(images)

            num_images = len(images)
            num_train = int(train_ratio * num_images)
            num_val = int(val_ratio * num_images)
            num_test = num_images - num_train - num_val

            num_test = int(test_ratio * num_images)
            if (num_train + num_val + num_test) > num_images:
                num_test = num_images - num_train - num_val
                if num_test < 0:
                    num_val = int(val_ratio * num_images)
                    num_train = num_images - num_val - num_test
                    if num_train < 0:
                        num_train = 0


            train_images = images[:num_train]
            val_images = images[num_train : num_train + num_val]
            test_images = images[num_train + num_val : num_train + num_val + num_test]

            os.makedirs(os.path.join(train_path, class_name), exist_ok=True)
            os.makedirs(os.path.join(val_path, class_name), exist_ok=True)
            os.makedirs(os.path.join(test_path, class_name), exist_ok=True)

            for img in train_images:
                shutil.move(os.path.join(class_source_path, img), os.path.join(train_path, class_name, img))
            for img in val_images:
                shutil.move(os.path.join(class_source_path, img), os.path.join(val_path, class_name, img))
            for img in test_images:
                shutil.move(os.path.join(class_source_path, img), os.path.join(test_path, class_name, img))

            if not os.listdir(class_source_path):
                os.rmdir(class_source_path)

        print("Dataset split into 'train', 'val', 'test' successfully.")

    elif has_train and has_test and not has_val:
        print("'train' and 'test' folders found, but 'val' is missing. Creating 'val' folder...")
        os.makedirs(val_path, exist_ok=True)

        print("Moving images from 'train' to create 'val' split...")
        train_classes = [name for name in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, name))]

        for class_name in train_classes:
            train_class_path = os.path.join(train_path, class_name)
            val_class_path = os.path.join(val_path, class_name)
            os.makedirs(val_class_path, exist_ok=True)

            images = [f for f in os.listdir(train_class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            random.shuffle(images)

            num_val_to_move = int(val_ratio / (train_ratio + val_ratio) * len(images))
            if num_val_to_move == 0 and len(images) > 0:
                num_val_to_move = 1

            val_images_to_move = images[:num_val_to_move]

            for img in val_images_to_move:
                shutil.move(os.path.join(train_class_path, img), os.path.join(val_class_path, img))
        print("'val' split created successfully.")

    else:
        print("Existing 'train', 'val', 'test' folders found. Proceeding with analysis.")

    split_totals = {}
    for split_name, current_split_path in [('train', train_path), ('val', val_path), ('test', test_path)]:
        if os.path.isdir(current_split_path):
            classes = [name for name in os.listdir(current_split_path) if os.path.isdir(os.path.join(current_split_path, name))]
            total_images = 0
            for class_name in classes:
                class_dir = os.path.join(current_split_path, class_name)
                if os.path.isdir(class_dir):
                    total_images += len([f for f in os.listdir(class_dir)
                                         if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])

            split_totals[split_name] = {'classes': len(classes), 'images': total_images}
        else:
            print(f"Warning: Split folder '{split_name.upper()}' not found after preparation. Skipping this split in summary.")

    print("\n--- Final Dataset Overview ---")
    for split, data in split_totals.items():
        print(f"{split.upper()} Split:")
        print(f"  Number of classes: {data['classes']}")
        print(f"  Total images: {data['images']}")
        print()

prepare_and_analyze_dataset(base_path)

In [ ]:
class_counts_overall = {}
if os.path.isdir(base_path):
    for split_name in ['train', 'val', 'test']:
        split_path = os.path.join(base_path, split_name)
        if os.path.isdir(split_path):
            for class_name in os.listdir(split_path):
                class_path = os.path.join(split_path, class_name)
                if os.path.isdir(class_path):
                    image_count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
                    class_counts_overall[class_name] = class_counts_overall.get(class_name, 0) + image_count

if class_counts_overall:
    min_count_overall = min(class_counts_overall.values()) if class_counts_overall else 0
    max_count_overall = max(class_counts_overall.values()) if class_counts_overall else 0
    class_counts_overall = dict(sorted(class_counts_overall.items(), key=lambda item: item[1], reverse=True))
    print("Total number of classes (overall):", len(class_counts_overall))
    print("Total number of images (overall):", sum(class_counts_overall.values()))
    print("Class-wise image counts (overall):")
    for class_name, count in class_counts_overall.items():
        print(f"  {class_name}: {count} images")
    print(f"\nMinimum number of images in a class (overall): {min_count_overall}")
    print(f"Maximum number of images in a class (overall): {max_count_overall}")
else:
    print(f"No class folders found within the split directories under '{base_path}'.")

In [ ]:
print("\n--- Defining Callbacks ---")
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    filepath=os.path.join(output_dir, 'flower_DenseNet121.keras'),
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

def lr_schedule(epoch, lr):
    if epoch < 5:
        return lr
    elif epoch < 10:
        return lr * 0.5
    elif epoch < 15:
        return lr * 0.2
    else:
        return lr * 0.1

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

callbacks = [early_stopping, model_checkpoint, lr_scheduler]
print("Callbacks defined: EarlyStopping, ModelCheckpoint, LearningRateScheduler")

In [ ]:
def create_dataframe_from_folder(base_dir):
    filepaths = []
    labels = []
    data_sets = []

    for split_name in ['train', 'test', 'val']:
        split_path = os.path.join(base_dir, split_name)
        if not os.path.isdir(split_path):
            print(f"Warning: Split folder '{split_path}' not found. Skipping this split.")
            continue

        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_path):
                for file in os.listdir(class_path):
                    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                        full_path = os.path.join(class_path, file)
                        relative_path = os.path.relpath(full_path, base_dir)

                        filepaths.append(relative_path)
                        labels.append(class_name)
                        data_sets.append(split_name)

    new_df = pd.DataFrame({
        'filepaths': filepaths,
        'labels': labels,
        'image_path': [os.path.join(base_dir, fp) for fp in filepaths],
        'data set': data_sets
    })

    return new_df

print(f"\n--- Scanning '{base_path}' to create initial DataFrame from folder structure ---")
df = create_dataframe_from_folder(base_path)

if df.empty:
    raise ValueError(f"No image files found in '{base_path}'. Please check your base_path and folder structure.")
print(f"DataFrame created with {len(df)} entries.")

In [ ]:
train_df_original = df[df['data set'] == 'train'].copy()
test_df_original = df[df['data set'] == 'test'].copy()
validation_df_original = df[df['data set'] == 'val'].copy()
print("DataFrames for train, validation, and test splits created.")

In [ ]:
train_df_original = train_df_original.rename(columns={'labels': 'label'})
test_df_original = test_df_original.rename(columns={'labels': 'label'})
validation_df_original = validation_df_original.rename(columns={'labels': 'label'})
print("DataFrames for train, validation, and test splits created and columns renamed.")

In [ ]:
print("\n--- Detailed Data Distribution (Text Summary) ---")

print("\nTRAIN Split:")
train_class_distribution = train_df_original['label'].value_counts().sort_index()
print(f"Number of classes: {len(train_class_distribution)}")
print(f"Total images: {len(train_df_original)}")
print("Class-wise image counts:")
for class_name, count in train_class_distribution.items():
    print(f"  {class_name}: {count} images")
print(f"Minimum images per class: {train_class_distribution.min()} images")
print(f"Maximum images per class: {train_class_distribution.max()} images")

In [ ]:
print("\nVAL Split:")
val_class_distribution = validation_df_original['label'].value_counts().sort_index()
print(f"Number of classes: {len(val_class_distribution)}")
print(f"Total images: {len(validation_df_original)}")
print("Class-wise image counts:")
for class_name, count in val_class_distribution.items():
    print(f"  {class_name}: {count} images")
print(f"Minimum images per class: {val_class_distribution.min()} images")
print(f"Maximum images per class: {val_class_distribution.max()} images")

In [ ]:
print("\nTEST Split:")
test_class_distribution = test_df_original['label'].value_counts().sort_index()
print(f"Number of classes: {len(test_class_distribution)}")
print(f"Total images: {len(test_df_original)}")
print("Class-wise image counts:")
for class_name, count in test_class_distribution.items():
    print(f"  {class_name}: {count} images")
print(f"Minimum images per class: {test_class_distribution.min()} images")
print(f"Maximum images per class: {test_class_distribution.max()} images")

In [ ]:
temp_datagen = ImageDataGenerator()
temp_generator = temp_datagen.flow_from_dataframe(
    dataframe=df.rename(columns={'labels': 'label'}),
    x_col='image_path',
    y_col='label',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
overall_class_distribution = df['labels'].value_counts()
plt.figure(figsize=(14, 7))
sns.barplot(x=overall_class_distribution.index, y=overall_class_distribution.values, palette='cubehelix', hue=overall_class_distribution.index, legend=False)
plt.title('Overall Distribution of Classes Across All Data (Train, Test, Val Combined)')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(rotation=90, fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
train_class_distribution = train_df_original['label'].value_counts()
plt.figure(figsize=(14, 7))
sns.barplot(x=train_class_distribution.index, y=train_class_distribution.values, palette='viridis', hue=train_class_distribution.index, legend=False)
plt.title('Distribution of Classes in Training Set')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(rotation=90, fontsize=8)
plt.tight_layout()
plt.show()
print("\nTraining Set Class Distribution (from plot data):")
print(train_class_distribution)

In [ ]:
val_class_distribution = validation_df_original['label'].value_counts()
plt.figure(figsize=(14, 7))
sns.barplot(x=val_class_distribution.index, y=val_class_distribution.values, palette='magma', hue=val_class_distribution.index, legend=False)
plt.title('Distribution of Classes in Validation Set')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(rotation=90, fontsize=8)
plt.tight_layout()
plt.show()
print("\nValidation Set Class Distribution (from plot data):")
print(val_class_distribution)

In [ ]:
test_class_distribution = test_df_original['label'].value_counts()
plt.figure(figsize=(14, 7))
sns.barplot(x=test_class_distribution.index, y=test_class_distribution.values, palette='cividis', hue=test_class_distribution.index, legend=False)
plt.title('Distribution of Classes in Test Set')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(rotation=90, fontsize=8)
plt.tight_layout()
plt.show()
print("\nTest Set Class Distribution (from plot data):")
print(test_class_distribution)

In [ ]:
data_volume = pd.DataFrame({
    'Dataset': ['Train', 'Test', 'Validation'],
    'Count': [len(train_df_original), len(test_df_original), len(validation_df_original)]
})
plt.figure(figsize=(8, 5))
sns.barplot(x='Dataset', y='Count', data=data_volume, palette='coolwarm', hue='Dataset', legend=False)
plt.title('Volume of Train, Test, and Validation Datasets')
plt.xlabel('Dataset Type')
plt.ylabel('Number of Samples')
plt.tight_layout()
plt.show()

In [ ]:
unique_classes_count = pd.DataFrame({
    'Dataset': ['Train', 'Test', 'Validation'],
    'Unique Classes': [train_df_original['label'].nunique(), test_df_original['label'].nunique(), validation_df_original['label'].nunique()]
})
plt.figure(figsize=(8, 5))
sns.barplot(x='Dataset', y='Unique Classes', data=unique_classes_count, palette='rocket', hue='Dataset', legend=False)
plt.title('Number of Unique Classes per Dataset Split')
plt.xlabel('Dataset Type')
plt.ylabel('Count of Unique Classes')
plt.tight_layout()
plt.show()

In [ ]:
dataset_type_distribution = df['data set'].value_counts()
plt.figure(figsize=(7, 5))
plt.pie(dataset_type_distribution, labels=dataset_type_distribution.index, autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))
plt.title('Distribution of Samples by Dataset Type')
plt.axis('equal')
plt.show()

In [ ]:
print("\n--- Class Distribution in Training Set (with counts) ---")

train_class_distribution = train_df_original['label'].value_counts().sort_index()

plt.figure(figsize=(14, 7))
ax = sns.barplot(x=train_class_distribution.index, y=train_class_distribution.values, palette='viridis', hue=train_class_distribution.index, legend=False)
plt.title('Distribution of Classes in Training Set')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()

for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 5), textcoords='offset points', fontsize=8)

plt.show()

In [ ]:
print("\n--- Visualizing Image File Size Distribution ---")

def get_file_size(image_path):
    try:
        return os.path.getsize(image_path)
    except Exception as e:
        logging.error(f"Could not get file size for {image_path}: {e}")
        return None

df['file_size'] = df['image_path'].apply(get_file_size)

df_with_sizes = df.dropna(subset=['file_size']).copy()

if not df_with_sizes.empty:
    df_with_sizes['file_size_kb'] = df_with_sizes['file_size'] / 1024


    plt.figure(figsize=(10, 6))
    sns.histplot(df_with_sizes['file_size_kb'], bins=50, kde=True)
    plt.title('Distribution of Image File Sizes (KB)')
    plt.xlabel('File Size (KB)')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

    print("\nSummary statistics for image file sizes (bytes):")
    print(df_with_sizes['file_size'].describe())
else:
     print("No image file size data could be retrieved to plot.")

In [ ]:
print("\n--- Visualizing Class Distribution by Split (Stacked Bar Chart) ---")

df_combined = pd.concat([
    train_df_original.assign(split='Train'),
    validation_df_original.assign(split='Validation'),
    test_df_original.assign(split='Test')
])

class_split_counts = pd.crosstab(df_combined['label'], df_combined['split'])

ax = class_split_counts.plot(kind='bar', stacked=True, figsize=(12, 7), colormap='viridis')

plt.title('Class Distribution by Dataset Split (Stacked Bar Chart)')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Dataset Split')
plt.tight_layout()
plt.show()

In [ ]:
print("\n--- Visualizing Image Dimensions Distribution ---")

def get_image_dimensions(image_path):
    try:
        with Image.open(image_path) as img:
            return img.size
    except Exception as e:
        logging.error(f"Could not get dimensions for {image_path}: {e}")
        return None

df['dimensions'] = df['image_path'].apply(get_image_dimensions)

df_with_dimensions = df.dropna(subset=['dimensions']).copy()

if not df_with_dimensions.empty:
    df_with_dimensions['width'] = df_with_dimensions['dimensions'].apply(lambda x: x[0])
    df_with_dimensions['height'] = df_with_dimensions['dimensions'].apply(lambda x: x[1])

    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    sns.histplot(df_with_dimensions['width'], bins=50, kde=True)
    plt.title('Distribution of Image Widths')
    plt.xlabel('Width (pixels)')
    plt.ylabel('Frequency')
    plt.grid(True)

    plt.subplot(1, 2, 2)
    sns.histplot(df_with_dimensions['height'], bins=50, kde=True)
    plt.title('Distribution of Image Heights')
    plt.xlabel('Height (pixels)')
    plt.ylabel('Frequency')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    print("\nSummary statistics for image dimensions:")
    print(df_with_dimensions[['width', 'height']].describe())

else:
    print("No image dimension data could be retrieved to plot.")

In [ ]:
def display_sample_images(dataframe, num_classes=5, images_per_class=3):
    unique_classes = dataframe['label'].unique()
    if len(unique_classes) < num_classes:
        num_classes = len(unique_classes)

    selected_classes = np.random.choice(unique_classes, num_classes, replace=False)

    plt.figure(figsize=(images_per_class * 3, num_classes * 3))
    plt.suptitle('Sample Images from Different Classes', fontsize=18, y=1.02)

    for i, class_name in enumerate(selected_classes):
        class_images = dataframe[dataframe['label'] == class_name]['image_path'].sample(min(images_per_class, len(dataframe[dataframe['label'] == class_name])), random_state=42)

        for j, image_path in enumerate(class_images):
            ax = plt.subplot(num_classes, images_per_class, i * images_per_class + j + 1)
            try:
                img = Image.open(image_path)
                ax.imshow(img)
                ax.set_title(f"{class_name}", fontsize=10)
                ax.axis('off')
            except Exception as e:
                ax.set_title(f"Error loading {class_name}", fontsize=10)
                ax.axis('off')
                logging.error(f"Could not load image {image_path}: {e}")

    plt.tight_layout(rect=[0, 0.03, 1, 0.98])
    plt.show()

print("\n--- Displaying Sample Images ---")
display_sample_images(train_df_original, num_classes=5, images_per_class=3)

In [ ]:
print("\n--- Setting up ImageDataGenerators ---")
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    vertical_flip=True,
    channel_shift_range=150,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df_original,
    x_col='image_path',
    y_col='label',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=RANDOM_SEED
)

validation_generator = val_test_datagen.flow_from_dataframe(
    dataframe=validation_df_original,
    x_col='image_path',
    y_col='label',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df_original,
    x_col='image_path',
    y_col='label',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
NUM_CLASSES = len(train_generator.class_indices)
print(f"\nFinal NUM_CLASSES for model output: {NUM_CLASSES}")

In [ ]:
print("\n--- Class Serialization (Class Name to Index Mapping) ---")
sorted_class_indices = sorted(train_generator.class_indices.items(), key=lambda item: item[1])
class_names = [item[0] for item in sorted_class_indices]
for class_name, index in sorted_class_indices:
    print(f"  {class_name}: {index}")

In [ ]:
def build_model(num_classes, input_shape=(IMG_WIDTH, IMG_HEIGHT, CHANNEL_NUM)):
    print(f"Building model with input_shape: {input_shape} and {num_classes} classes.")
    base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')

    base_model.trainable = False

    print("InceptionResNetV2 base model loaded and frozen.")

    x = base_model.output

    x = Dense(1024, kernel_regularizer=regularizers.l2(0.0001))(x)
    x = LeakyReLU(negative_slope=0.01)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(512, kernel_regularizer=regularizers.l2(0.0001))(x)
    x = LeakyReLU(negative_slope=0.01)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    x = Dense(256, kernel_regularizer=regularizers.l2(0.0001))(x)
    x = LeakyReLU(negative_slope=0.01)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(128, kernel_regularizer=regularizers.l2(0.0001))(x)
    x = LeakyReLU(negative_slope=0.01)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    print("Custom classification head added.")
    return model, base_model

print("\n--- Building the Model ---")
model, base_model = build_model(NUM_CLASSES, input_shape)
print("Model built successfully.")

In [ ]:
print("\n--- Compiling the Model ---")
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)
print("Model compiled successfully.")

In [ ]:
model.summary()

In [ ]:
print("\n--- Starting Model Training (Head Only, Base Frozen) ---")
try:
    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=validation_generator,
        callbacks=callbacks,
        verbose=1
    )
    print("\n--- Model Training Finished ---")
except Exception as e:
    print(f"Error during model training: {e}")
    history = None

In [ ]:
print("\n--- Evaluating Model on Test Set ---")
if history:
    test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
else:
    print("Model training failed, skipping evaluation.")

In [ ]:
print("\n--- Plotting Learning Rate over Epochs ---")

if history and 'lr' in history.history:
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['lr'])
    plt.title('Learning Rate over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.grid(True)
    plt.show()
else:
    print("Learning rate history not available. Please ensure the LearningRateScheduler callback was used and training completed successfully.")

In [ ]:
print("\n--- Generating Performance Plots ---")

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
print("\n--- Generating Confusion Matrix ---")

test_generator.reset()
Y_pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
y_pred_classes = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes
conf_matrix = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n--- Generating Classification Report ---")
report = classification_report(y_true, y_pred_classes, target_names=class_names, zero_division=0)
print(report)

In [ ]:
print("\n--- Generating ROC Curve and AUC (Micro-average) ---")

test_generator.reset()
Y_pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
y_true = test_generator.classes
y_true_one_hot = to_categorical(y_true, num_classes=NUM_CLASSES)

fpr, tpr, _ = roc_curve(y_true_one_hot.ravel(), Y_pred.ravel())
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Micro-average Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
print("\n--- Generating ROC Curves for Each Class (One-vs-Rest) ---")
plt.figure(figsize=(12, 10))
colors = cycle(['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'black', 'purple', 'pink', 'brown'])
for i, color in zip(range(NUM_CLASSES), colors):
    fpr, tpr, _ = roc_curve(y_true_one_hot[:, i], Y_pred[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=color, lw=2,
              label='ROC curve of class {0} (area = {1:0.2f})'.format(class_names[i], roc_auc))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Each Class')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
print("\n--- Generating Precision-Recall Curve and Average Precision (Micro-average) ---")

test_generator.reset()
Y_pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
y_true = test_generator.classes
y_true_one_hot = to_categorical(y_true, num_classes=NUM_CLASSES)

precision, recall, _ = precision_recall_curve(y_true_one_hot.ravel(), Y_pred.ravel())
average_precision = average_precision_score(y_true_one_hot, Y_pred, average="micro")

plt.figure(figsize=(10, 8))
plt.plot(recall, precision, color='darkorange', lw=2, label=f'Precision-Recall curve (area = {average_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Micro-average Precision-Recall Curve')
plt.legend(loc="lower left")
plt.grid(True)
plt.show()

In [ ]:
print("\n--- Generating Precision-Recall Curves for Each Class (One-vs-Rest) ---")
plt.figure(figsize=(12, 10))
colors = cycle(['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'black', 'purple', 'pink', 'brown'])
for i, color in zip(range(NUM_CLASSES), colors):
    precision, recall, _ = precision_recall_curve(y_true_one_hot[:, i], Y_pred[:, i])
    average_precision = average_precision_score(y_true_one_hot[:, i], Y_pred[:, i])
    plt.plot(recall, precision, color=color, lw=2,
              label='Precision-Recall curve of class {0} (area = {1:0.2f})'.format(class_names[i], average_precision))

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for Each Class')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()